In [ ]:
# setup paths
import sys
basedir = '/Users/nickm/thesis/InvariantPreservingMLSolvers/1d_euler'
readwritedir = '/Users/nickm/thesis/InvariantPreservingMLSolvers/1d_euler'

sys.path.append('{}/core'.format(basedir))
sys.path.append('{}/simulate'.format(basedir))
sys.path.append('{}/ml'.format(basedir))

In [ ]:
# import external packages
import jax
import jax.numpy as jnp
import numpy as onp
from jax import config, vmap
config.update("jax_enable_x64", True)
config.update("jax_disable_jit", True)
import xarray
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from initialconditions import get_a0, f_init_lax_shock_tube, shock_tube_problem_1, shock_tube_problem_2, shock_tube_problem_3, shock_tube_problem_4, shock_tube_problem_5, shock_tube_problem_6, shock_tube_problem_7, shock_tube_problem_8, shock_tube_blast_wave
from simparams import CoreParams, SimulationParams
from simulations import EulerFVSim
from helper import get_rho, get_u, get_p, get_c
from trajectory import get_trajectory_fn, get_inner_fn

In [ ]:
def get_core_params(Lx = 1.0, gamma = 5/3, bc = 'periodic', fluxstr = 'laxfriedrichs'):
    return CoreParams(Lx, gamma, bc, fluxstr)


def get_sim_params(name = "test", cfl_safety=0.3, rk='ssp_rk3'):
    return SimulationParams(name, basedir, readwritedir, cfl_safety, rk)


def plot_a(a, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3, title = ""):
    x = jnp.linspace(0.0, core_params.Lx, a.shape[1])
    
    fig, axs = plt.subplots(1, 3, figsize=(11, 3))
    axs[0].plot(x, get_rho(a, core_params))
    axs[0].set_ylabel(r'$\rho$')
    axs[0].set_ylim([mins[0], maxs[0]])
    
    axs[1].plot(x, get_u(a, core_params))
    axs[1].set_ylabel(r'$u$')
    axs[1].set_ylim([mins[1], maxs[1]])
    
    axs[2].plot(x, get_p(a, core_params))
    axs[2].set_ylabel(r'$p$')
    axs[2].set_ylim([mins[2], maxs[2]])
    
    fig.suptitle(title)
    fig.tight_layout()

def plot_ac(a, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    x = jnp.linspace(0.0, core_params.Lx, a.shape[1])
    
    fig, axs = plt.subplots(1, 4, figsize=(11, 3))
    axs[0].plot(x, get_rho(a, core_params))
    axs[0].set_title(r'$\rho$')
    axs[0].set_ylim([mins[0], maxs[0]])
    
    axs[1].plot(x, get_u(a, core_params))
    axs[1].set_ylabel(r'$u$')
    axs[1].set_ylim([mins[1], maxs[1]])
    
    axs[2].plot(x, get_p(a, core_params))
    axs[2].set_ylabel(r'$p$')
    axs[2].set_ylim([mins[2], maxs[2]])
    
    axs[3].plot(x, get_c(a, core_params))
    axs[3].set_ylabel(r'$c$')
    axs[3].set_ylim([mins[1], maxs[1]])

def plot_trajectory(trajectory, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    nx = trajectory.shape[2]
    xs = jnp.arange(nx) * core_params.Lx / nx
    xs = xs.T.reshape(-1)
    coords = {
        'x': xs,
        'time': t_inner * jnp.arange(outer_steps)
    }
    rhos = trajectory[:, 0, :]
    xarray.DataArray(rhos, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

    us = trajectory[:, 1, :] / trajectory[:, 0, :]
    xarray.DataArray(us, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

    ps = (core_params.gamma - 1) * (trajectory[:, 2, :] - 0.5 * trajectory[:, 1, :]**2 / trajectory[:, 0, :])
    xarray.DataArray(ps, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

In [ ]:
def get_trajectories(a0, t_inner, outer_step, Lx=1.0, gamma=1.4, flux_1 = 'musclconserved', flux_2 = 'musclprimitive', flux_3 = 'musclcharacteristic'):
    
    BC = 'ghost'
    IP = False
    CFL = 0.3
    RK = 'ssp_rk3'
    
    kwargs_core_params_conserved = {'Lx': Lx, 'gamma': gamma, 'bc': BC, 'fluxstr': flux_1}
    kwargs_core_params_primitive = {'Lx': Lx, 'gamma': gamma, 'bc': BC, 'fluxstr': flux_2}
    kwargs_core_params_characteristic = {'Lx': Lx, 'gamma': gamma, 'bc': BC, 'fluxstr': flux_3}
    kwargs_sim = {'name' : "test_euler", 'cfl_safety' : CFL, 'rk' : RK}

    core_params_conserved = get_core_params(**kwargs_core_params_conserved)
    core_params_primitive = get_core_params(**kwargs_core_params_primitive)
    core_params_characteristic = get_core_params(**kwargs_core_params_characteristic)
    
    sim_params = get_sim_params(**kwargs_sim)
    sim_conserved = EulerFVSim(core_params_conserved, sim_params, invariant_preserving=IP)
    sim_primitive = EulerFVSim(core_params_primitive, sim_params, invariant_preserving=IP)
    sim_characteristic = EulerFVSim(core_params_characteristic, sim_params, invariant_preserving=IP)
    
    inner_fn_conserved = get_inner_fn(sim_conserved.step_fn, sim_conserved.dt_fn, t_inner)
    trajectory_fn_conserved = get_trajectory_fn(inner_fn_conserved, outer_steps)
    trajectory_conserved = trajectory_fn_conserved(a0)
    
    inner_fn_primitive = get_inner_fn(sim_primitive.step_fn, sim_primitive.dt_fn, t_inner)
    trajectory_fn_primitive = get_trajectory_fn(inner_fn_primitive, outer_steps)
    trajectory_primitive = trajectory_fn_primitive(a0)

    inner_fn_characteristic = get_inner_fn(sim_characteristic.step_fn, sim_characteristic.dt_fn, t_inner)
    trajectory_fn_characteristic = get_trajectory_fn(inner_fn_characteristic, outer_steps)
    trajectory_characteristic = trajectory_fn_characteristic(a0)
    
    return trajectory_conserved, trajectory_primitive, trajectory_characteristic

### Shock Tube Problem 1

JE2: http://ammar-hakim.org/sj/je/je2/je2-euler-shock.html

In [ ]:
nx = 100
Lx = 1.0
gamma = 1.4
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_problem_1(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 0.05
outer_steps = 5

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps)

In [ ]:
maxs = [1.1, 1.5, 1.1]
mins = [-0.05, -0.05, -0.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Problem 2

http://ammar-hakim.org/sj/je/je2/je2-euler-shock.html

MUSCL gives NaNs for this problem without a positivity fix.

In [ ]:
nx = 100
Lx = 1.0
gamma = 1.4
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_problem_2(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 0.015
outer_steps = 10

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps)

In [ ]:
maxs = [1.1, 2.1, 0.5]
mins = [-0.05, -2.1, -0.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)

t=1
plot_a(traj_cons[t], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[t], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[t], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Problem 3

In [ ]:
nx = 100
Lx = 1.0
gamma = 5/3
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_problem_3(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 1.0
outer_steps = 2

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps, gamma=gamma)

In [ ]:
maxs = [4.6, 1.5, 1.5]
mins = [-0.05, -1.5, -0.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Problem 4

In [ ]:
nx = 100
Lx = 1.0
gamma = 1.4
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_problem_4(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 0.012
outer_steps = 2

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps)

In [ ]:
maxs = [6.5, 5.5, 1050.1]
mins = [-0.05, -20.5, -20.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Problem 5

In [ ]:
nx = 100
Lx = 1.0
gamma = 1.4
f_init = shock_tube_problem_5(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 0.035
outer_steps = 2

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps)

In [ ]:
maxs = [35.0, 20.5, 1800.0]
mins = [-1.0, -10.0, -1.0]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Problem 6

In [ ]:
nx = 100
Lx = 1.0
gamma = 1.4
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_problem_6(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 1.0
outer_steps = 3

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps)

In [ ]:
maxs = [1.5, 1.5, 1.1]
mins = [-0.05, -0.05, -0.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Problem 7: Density Propagation

In [ ]:
nx = 100
Lx = 1.0
gamma = 1.4
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_problem_7(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 1.0
outer_steps = 3

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps)

In [ ]:
maxs = [1.5, 1.5, 1.1]
mins = [-0.05, -0.05, -0.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Problem 8

In [ ]:
nx = 800
Lx = 0.5
gamma = 1.4
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_problem_8(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 0.0039
outer_steps = 2

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps, Lx = Lx)

In [ ]:
maxs = [41.0, 16.0, 800.0]
mins = [-0.05, -0.05, -0.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")

### Woodward-Collela blast wave problem

In [ ]:
nx = 400
Lx = 1.0
gamma = 1.4
kwargs_core_params = {'Lx': Lx, 'gamma': gamma, 'bc': 'ghost', 'fluxstr': 'roe'}
core_params = get_core_params(**kwargs_core_params)
f_init = shock_tube_blast_wave(core_params)
a0 = get_a0(f_init, core_params, nx)
t_inner = 0.0038
outer_steps = 11

traj_cons, traj_prim, traj_char = get_trajectories(a0, t_inner, outer_steps, Lx = Lx)

In [ ]:
maxs = [8, 20.0, 1050.0]
mins = [-0.05, -2.0, -5.00]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(traj_cons[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Conserved Variables")
plot_a(traj_prim[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Primitive Variables")
plot_a(traj_char[-1], core_params, maxs=maxs, mins=mins, title="Reconstruction in Characteristic Variables")